In [20]:
import os
from urllib.parse import urlencode 
import pandas as pd 
from IPython.display import Image, display, HTML
import ipywidgets as widgets 
from ipywidgets import interact, interact_manual 
from chart_studio.plotly import iplot 
from astropy import units as u 
from astropy.coordinates import SkyCoord
from astropy.time import Time, TimeDelta
from astroquery.cadc import Cadc 
from datetime import datetime
from glue_jupyter.data import require_data
import glue_jupyter as gj
import csv

params = {'claimedtype' : 'Ia', 'format' : 'csv' }
Catalog_domain ='https://api.sne.space/catalog'
query_str  = urlencode(params)
url = '{}?{}&first'.format(Catalog_domain,query_str)
data = pd.read_csv(url)


def using_adql():
    global results
    query_outline = """SELECT {num} * FROM caom2.Plane AS Plane 
    JOIN caom2.Observation AS Observation ON Plane.obsID = Observation.obsID
    WHERE (INTERSECTS( INTERVAL( {mjd_start}, {mjd_end} ), Plane.time_bounds_samples ) = 1 
    AND CONTAINS( POINT('ICRS', {ra}, {dec}), Plane.position_bounds ) = 1 
    AND LOWER(Plane.energy_bandpassName) LIKE '{filter}%' 
    AND collection = '{collection}'
    AND calibrationLevel >= {cal_level}
    AND (Plane.quality_flag IS NULL OR Plane.quality_flag != 'junk'))
    ORDER BY time_bounds_lower {order}"""
    
    query_params = {
    'num': '',  # Restricts the number of results (empty string returns all)
    'mjd_start': start_date.mjd,
    'mjd_end': end_date.mjd,
    'ra': coords.ra.degree,
    'dec': coords.dec.degree,
    'filter': 'r',
    'collection': 'CFHT',
    'cal_level': 2,
    'order': 'ASC'  # Order the results from oldest to newest        
    }
    
    cadc = Cadc() 
    results = cadc.exec_sync(query_outline.format(**query_params))
    columns_subset = ['productID', 'collection', 'energy_bandpassName', 'time_bounds_samples','time_bounds_lower', 'time_exposure']
    
    app = gj .jglue()
    with open("catalog.csv", "w") as f:
        f_writer = csv.writer(f)
        f_writer.writerow([i for i in results[columns_subset].columns])
        f_writer.writerows(results[columns_subset])
        
        
    sn_data = app.load_data('catalog.csv')
    scatter_viewer = app.scatter2d(x = 'productID',y = 'time_bounds_lower', data = sn_data)
    histogram_viewer = app.histogram1d(x = 'productID', data = sn_data)
    




def select_supernova(supernova):
    global start_date, end_date, coords
    superN = supernova.split()
    selected_superNova = sn_data.loc[sn_data.name == superN[0]].squeeze()
    sn_name = selected_superNova['name']
    sn_ra = selected_superNova['ra']
    sn_dec =  selected_superNova['dec']
    sn_maxdate = selected_superNova['maxdate']

    coords = SkyCoord(sn_ra, sn_dec, unit=(u.hourangle, u.deg))
    days_before = 20.0
    days_after = 100.0 
    maxdate = Time(sn_maxdate.replace('/','-'), format='isot')
    start_date = maxdate - TimeDelta(days_before, format='jd')
    end_date = maxdate + TimeDelta(days_after, format='jd')
    interact(using_adql )

@interact
def filter_data(z_min =0.2, z_max =0.3, event =['SNLS'] ):
    global sn_data
    sn_data = data.copy()
    sn_data = sn_data.dropna(subset = ['discoverdate', 'maxdate','redshift'])
    sn_data = sn_data[(sn_data['redshift']>z_min) & (sn_data['redshift']<z_max)] 
    sn_data = sn_data[sn_data['event'].str.startswith(event)]
    sn_data = sn_data[sn_data['discoverdate']< sn_data['maxdate']]
    print('Query url: {}'.format(url))
    print('Number of results: {}'.format(len(sn_data)))
    print(' '*5+'Name'+' '*8+ 'claimedtype'+' '*5+ 'catalog')
    interact(select_supernova, supernova = (sn_data['name']+' '*8+sn_data['claimedtype']+' '*12+sn_data['catalog']).tolist())


interactive(children=(FloatSlider(value=0.2, description='z_min', max=0.6000000000000001, min=-0.2), FloatSlid…